In [1]:
simbolo_test = "BTG"
n_simboli_addestramento = 50
epochs = 50
batch_size = 32
learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo
seed = 10 
elenco_features = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "Volume",
    "MACDh",
    "PSAR",
    "PSARaf",
    "SUPERT", 
    "TRIX"
]
elenco_target = [
        "EMA_5",
        "EMA_20", 
        "EMA_50"    
]

def crea_modello(giorni_previsione, n_targets, n_features):
    model = Sequential([
        LSTM(50, input_shape=(n_timesteps, n_features)),
        RepeatVector(giorni_previsione),
        LSTM(50, return_sequences=True, kernel_regularizer=l2(0)),
        #LSTM(50, return_sequences=True, kernel_regularizer=l2(0)),
        #Dropout(0.2),
        #LSTM(100, return_sequences=True, recurrent_initializer="glorot_uniform", kernel_initializer="uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #LSTM(50, recurrent_initializer="glorot_uniform", kernel_initializer="uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        #Dense(64, activation='relu', kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.01)),
        #Dropout(0.2),
        TimeDistributed(Dense(n_targets, activation='linear'))
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error', metrics=['mae'])
    return model

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.base import clone
scaler = PowerTransformer()
#scaler = RobustScaler()

In [2]:
print("Importa librerie", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import fx_com
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l1, l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
from tensorflow.keras.optimizers import Adam
print(".", end="", flush=True)
from sklearn.model_selection import train_test_split
print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
import os
print(".", end="\n", flush=True)

np.random.seed(seed)
tf.random.set_seed(seed)
pd.set_option("display.max_columns", None)

col_features = {col: idx for idx, col in enumerate(elenco_features)}
col_targets = {col: idx for idx, col in enumerate(elenco_target)}
n_features = len(col_features)
n_targets = len(col_targets)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[lista_ticker["Ticker"] != simbolo_test, :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(giorni_previsione, n_targets, n_features)

def to_XY(dati_ticker, n_timesteps, giorni_previsione, addestramento=True):
    new_dates = pd.bdate_range(start=dati_ticker.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates)
    dati_ticker = pd.concat([dati_ticker, df_new])

    features = dati_ticker[elenco_features]
    targets = dati_ticker[elenco_target]

    if addestramento:
        i_tot = len(features) - giorni_previsione*2
    else:
        i_tot = len(features) - giorni_previsione
    X, Y = [], []
    for i in range(n_timesteps - 1, i_tot):
        X.append(features.iloc[i - (n_timesteps - 1):i + 1])
        Y.append(targets.iloc[i + 1:i + 1 + giorni_previsione])
    
    idx = dati_ticker.index[n_timesteps - 1:i_tot]
    
    return idx, np.array(X), np.array(Y)


Importa librerie................
Download lista ticker
Caricamento modello esistente


In [3]:
for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
    if ticker["Close"].iloc[-1] >= 1:
        ticker.index = ticker.index.date
        ticker = fx.crea_indicatori(ticker)
        ticker.dropna(axis=0, inplace=True)

        print("Definizione features e target")
        idx, X, Y = to_XY(ticker, n_timesteps, giorni_previsione)

        print("Scaler")
        X = X.reshape((-1, n_timesteps * n_features))
        Y = Y.reshape((-1, giorni_previsione * n_targets))

        #X_train, X_test, Y_train, Y_test, idx_train, idx_test = train_test_split(X, Y, idx, test_size=0.2, random_state=seed)
        X_train = X
        Y_train = Y

        X_scaler = clone(scaler)
        Y_scaler = clone(scaler)
        X_scaler.fit(X_train)
        Y_scaler.fit(Y_train)
        X_train = X_scaler.fit_transform(X_train)
        #X_test = X_scaler.fit_transform(X_test)
        Y_train = Y_scaler.fit_transform(Y_train)
        #Y_test = Y_scaler.fit_transform(Y_test)

        X_train = X_train.reshape((-1, n_timesteps, n_features))
        #X_test = X_test.reshape((-1, n_timesteps, n_features))
        Y_train = Y_train.reshape((-1, giorni_previsione, n_targets))
        #Y_test = Y_test.reshape((-1, giorni_previsione, n_targets))

        print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
        model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)#, validation_data=(X_test, Y_test))
        model.save("LSTM.keras")


0 di 50: Ticker RLX
Download dati ticker
Definizione features e target
Scaler
Addestramento modello epochs=50, batch_size=32
Epoch 1/50
16/16 [==============================] - 4s 27ms/step - loss: 0.0344 - mae: 0.1193
Epoch 2/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0107 - mae: 0.0731
Epoch 3/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0072 - mae: 0.0601
Epoch 4/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0054 - mae: 0.0524
Epoch 5/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0046 - mae: 0.0476
Epoch 6/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0041 - mae: 0.0450
Epoch 7/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0038 - mae: 0.0429
Epoch 8/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0035 - mae: 0.0409
Epoch 9/50
16/16 [==============================] - 0s 27ms/step - loss: 0.0033 - mae: 0.0394
Epoch 10/50
16/16 [==========

103/103 [==============================] - 6s 59ms/step - loss: 0.0013 - mae: 0.0231
Epoch 15/50
103/103 [==============================] - 6s 54ms/step - loss: 0.0012 - mae: 0.0234
Epoch 16/50
103/103 [==============================] - 6s 54ms/step - loss: 0.0011 - mae: 0.0219
Epoch 17/50
103/103 [==============================] - 6s 54ms/step - loss: 0.0011 - mae: 0.0219
Epoch 18/50
103/103 [==============================] - 6s 54ms/step - loss: 0.0011 - mae: 0.0219
Epoch 19/50
103/103 [==============================] - 6s 55ms/step - loss: 0.0010 - mae: 0.0212
Epoch 20/50
103/103 [==============================] - 6s 54ms/step - loss: 9.4458e-04 - mae: 0.0205
Epoch 21/50
103/103 [==============================] - 6s 53ms/step - loss: 9.3600e-04 - mae: 0.0202
Epoch 22/50
103/103 [==============================] - 6s 54ms/step - loss: 9.4574e-04 - mae: 0.0204
Epoch 23/50
103/103 [==============================] - 5s 52ms/step - loss: 8.6197e-04 - mae: 0.0196
Epoch 24/50
103/103 [=====

10/10 [==============================] - 1s 53ms/step - loss: 0.0023 - mae: 0.0324
Epoch 29/50
10/10 [==============================] - 0s 47ms/step - loss: 0.0022 - mae: 0.0320
Epoch 30/50
10/10 [==============================] - 0s 49ms/step - loss: 0.0021 - mae: 0.0315
Epoch 31/50
10/10 [==============================] - 0s 47ms/step - loss: 0.0021 - mae: 0.0311
Epoch 32/50
10/10 [==============================] - 1s 50ms/step - loss: 0.0022 - mae: 0.0318
Epoch 33/50
10/10 [==============================] - 0s 47ms/step - loss: 0.0022 - mae: 0.0323
Epoch 34/50
10/10 [==============================] - 0s 50ms/step - loss: 0.0022 - mae: 0.0322
Epoch 35/50
10/10 [==============================] - 0s 48ms/step - loss: 0.0022 - mae: 0.0323
Epoch 36/50
10/10 [==============================] - 0s 48ms/step - loss: 0.0020 - mae: 0.0308
Epoch 37/50
10/10 [==============================] - 0s 46ms/step - loss: 0.0019 - mae: 0.0297
Epoch 38/50
10/10 [==============================] - 0s 46ms/s

Epoch 9/50
103/103 [==============================] - 5s 48ms/step - loss: 0.0022 - mae: 0.0304
Epoch 10/50
103/103 [==============================] - 5s 49ms/step - loss: 0.0022 - mae: 0.0300
Epoch 11/50
103/103 [==============================] - 5s 48ms/step - loss: 0.0021 - mae: 0.0288
Epoch 12/50
103/103 [==============================] - 5s 49ms/step - loss: 0.0019 - mae: 0.0279
Epoch 13/50
103/103 [==============================] - 5s 47ms/step - loss: 0.0018 - mae: 0.0273
Epoch 14/50
103/103 [==============================] - 5s 46ms/step - loss: 0.0018 - mae: 0.0274
Epoch 15/50
103/103 [==============================] - 5s 47ms/step - loss: 0.0017 - mae: 0.0263
Epoch 16/50
103/103 [==============================] - 5s 47ms/step - loss: 0.0016 - mae: 0.0259
Epoch 17/50
103/103 [==============================] - 5s 47ms/step - loss: 0.0015 - mae: 0.0251
Epoch 18/50
103/103 [==============================] - 5s 47ms/step - loss: 0.0016 - mae: 0.0258
Epoch 19/50
103/103 [==========

103/103 [==============================] - 5s 51ms/step - loss: 9.0460e-04 - mae: 0.0201
Epoch 21/50
103/103 [==============================] - 5s 50ms/step - loss: 8.6403e-04 - mae: 0.0196
Epoch 22/50
103/103 [==============================] - 5s 50ms/step - loss: 8.3054e-04 - mae: 0.0192
Epoch 23/50
103/103 [==============================] - 5s 50ms/step - loss: 8.1059e-04 - mae: 0.0189
Epoch 24/50
103/103 [==============================] - 5s 50ms/step - loss: 0.0011 - mae: 0.0208
Epoch 25/50
103/103 [==============================] - 5s 51ms/step - loss: 9.6774e-04 - mae: 0.0200
Epoch 26/50
103/103 [==============================] - 5s 50ms/step - loss: 9.7407e-04 - mae: 0.0198
Epoch 27/50
103/103 [==============================] - 5s 50ms/step - loss: 0.0012 - mae: 0.0224
Epoch 28/50
103/103 [==============================] - 5s 50ms/step - loss: 8.5212e-04 - mae: 0.0193
Epoch 29/50
103/103 [==============================] - 5s 50ms/step - loss: 7.8471e-04 - mae: 0.0186
Epoch 30/5

...

20/20 [==============================] - 1s 49ms/step - loss: 0.0014 - mae: 0.0254
Epoch 24/50
20/20 [==============================] - 1s 47ms/step - loss: 0.0014 - mae: 0.0261
Epoch 25/50
20/20 [==============================] - 1s 46ms/step - loss: 0.0013 - mae: 0.0247
Epoch 26/50
20/20 [==============================] - 1s 46ms/step - loss: 0.0013 - mae: 0.0254
Epoch 27/50
20/20 [==============================] - 1s 45ms/step - loss: 0.0013 - mae: 0.0244
Epoch 28/50
20/20 [==============================] - 1s 45ms/step - loss: 0.0012 - mae: 0.0236
Epoch 29/50
20/20 [==============================] - 1s 44ms/step - loss: 0.0012 - mae: 0.0232
Epoch 30/50
20/20 [==============================] - 1s 45ms/step - loss: 0.0012 - mae: 0.0236
Epoch 31/50
20/20 [==============================] - 1s 44ms/step - loss: 0.0012 - mae: 0.0234
Epoch 32/50
20/20 [==============================] - 1s 44ms/step - loss: 0.0011 - mae: 0.0229
Epoch 33/50
20/20 [==============================] - 1s 45ms/s

26/26 [==============================] - 1s 44ms/step - loss: 0.0014 - mae: 0.0255
Epoch 41/50
26/26 [==============================] - 1s 44ms/step - loss: 0.0014 - mae: 0.0250
Epoch 42/50
26/26 [==============================] - 1s 44ms/step - loss: 0.0014 - mae: 0.0254
Epoch 43/50
26/26 [==============================] - 1s 44ms/step - loss: 0.0013 - mae: 0.0249
Epoch 44/50
26/26 [==============================] - 1s 44ms/step - loss: 0.0013 - mae: 0.0247
Epoch 45/50
26/26 [==============================] - 1s 45ms/step - loss: 0.0014 - mae: 0.0250
Epoch 46/50
26/26 [==============================] - 1s 46ms/step - loss: 0.0013 - mae: 0.0247
Epoch 47/50
26/26 [==============================] - 1s 45ms/step - loss: 0.0013 - mae: 0.0242
Epoch 48/50
26/26 [==============================] - 1s 45ms/step - loss: 0.0013 - mae: 0.0242
Epoch 49/50
26/26 [==============================] - 1s 45ms/step - loss: 0.0013 - mae: 0.0240
Epoch 50/50
26/26 [==============================] - 1s 45ms/s

32/32 [==============================] - 1s 46ms/step - loss: 0.0063 - mae: 0.0447
Epoch 3/50
32/32 [==============================] - 1s 44ms/step - loss: 0.0072 - mae: 0.0410
Epoch 4/50
32/32 [==============================] - 1s 45ms/step - loss: 0.0042 - mae: 0.0349
Epoch 5/50
32/32 [==============================] - 1s 45ms/step - loss: 0.0033 - mae: 0.0333
Epoch 6/50
32/32 [==============================] - 1s 45ms/step - loss: 0.0025 - mae: 0.0302
Epoch 7/50
32/32 [==============================] - 1s 44ms/step - loss: 0.0021 - mae: 0.0283
Epoch 8/50
32/32 [==============================] - 1s 46ms/step - loss: 0.0020 - mae: 0.0269
Epoch 9/50
32/32 [==============================] - 1s 44ms/step - loss: 0.0018 - mae: 0.0262
Epoch 10/50
32/32 [==============================] - 1s 45ms/step - loss: 0.0017 - mae: 0.0243
Epoch 11/50
32/32 [==============================] - 1s 45ms/step - loss: 0.0014 - mae: 0.0235
Epoch 12/50
32/32 [==============================] - 1s 46ms/step - l

103/103 [==============================] - 7s 68ms/step - loss: 0.0020 - mae: 0.0286
Epoch 18/50
103/103 [==============================] - 6s 54ms/step - loss: 0.0019 - mae: 0.0278
Epoch 19/50
103/103 [==============================] - 5s 49ms/step - loss: 0.0019 - mae: 0.0281
Epoch 20/50
103/103 [==============================] - 5s 51ms/step - loss: 0.0020 - mae: 0.0283
Epoch 21/50
103/103 [==============================] - 5s 52ms/step - loss: 0.0018 - mae: 0.0270
Epoch 22/50
103/103 [==============================] - 5s 50ms/step - loss: 0.0018 - mae: 0.0272
Epoch 23/50
103/103 [==============================] - 5s 48ms/step - loss: 0.0020 - mae: 0.0273
Epoch 24/50
103/103 [==============================] - 5s 48ms/step - loss: 0.0018 - mae: 0.0267
Epoch 25/50
103/103 [==============================] - 5s 49ms/step - loss: 0.0016 - mae: 0.0258
Epoch 26/50
103/103 [==============================] - 5s 51ms/step - loss: 0.0015 - mae: 0.0246
Epoch 27/50
103/103 [=====================

ValueError: Found array with 0 sample(s) (shape=(0, 780)) while a minimum of 1 is required by PowerTransformer.

In [4]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev = to_XY(dati_previsione, n_timesteps, giorni_previsione, addestramento=False)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione * n_targets)
X_scaler_prev = clone(scaler)
Y_scaler_prev = clone(scaler)
X_prev = X_scaler_prev.fit_transform(X_prev)
Y_prev = Y_scaler_prev.fit_transform(Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
Y_prev = Y_prev.reshape(-1, giorni_previsione, n_targets)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)

X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = X_scaler_prev.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)

pred = pred.reshape(-1, giorni_previsione * n_targets)
pred = Y_scaler_prev.inverse_transform(pred)
pred = pred.reshape(-1, giorni_previsione, n_targets)



Download dati ticker per previsione
106/106 [==============================] - 2s 11ms/step


In [7]:
dati_previsione = dati_previsione.loc[indice_prev]
new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
df_new = pd.DataFrame(index=new_dates.date)
dati_previsione = pd.concat([dati_previsione, df_new])

col_analisi = "EMA_20"
risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)

In [8]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

target = go.Scatter(
    x = risultato.index,
    y = risultato[col_analisi],
    mode = 'lines',
    line = dict(color='rgba(0, 0, 0, .9)'),
    name = col_analisi
)

previsione = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'],
    mode = 'lines',
    line = dict(color='rgba(0, 0, 250, .9)'),
    name = 'Previsione'
)
'''
err_meno = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'] - err, 
    mode = 'lines',
    line = dict(
        color='rgba(0, 0, 250, .9)',
        width=1
    ),
    connectgaps = False,
    name = 'err-'
)                
err_piu = go.Scatter(
    x = risultato.index,
    y = risultato['Previsione'] + err, 
    mode = 'lines',
    line = dict(
        color='rgba(0, 0, 250, .9)',
        width=1
    ),
    fill='tonexty',
    fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
    connectgaps = False,
    name = 'err+'
)
'''
layout = dict(xaxis = dict(autorange=True),
              yaxis = dict(title = 'Previsioni', autorange=True),
              autosize = True,
              margin = go.layout.Margin(
                  l=0,  # Sinistra
                  r=0,  # Destra
                  b=0,  # Basso
                  t=50,  # Alto
                  pad=0  # Padding
              ),
              legend = dict(traceorder = 'normal', bordercolor = 'black')
)
fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.update_layout(layout)
fig.add_trace(target, row=1, col=1)
fig.add_trace(previsione, row=1, col=1)
#fig.add_trace(err_meno, row=1, col=1)
#fig.add_trace(err_piu, row=1, col=1)
pyo.plot(fig, filename="regressione.html", auto_open=True)



'regressione.html'